# Data: [Spotify](https://github.com/rfordatascience/tidytuesday/blob/master/data/2020/2020-01-21/readme.md)

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2020/2020-01-21/spotify_songs.csv')

## 1NF

In [3]:
# No repeating groups of columns

In [4]:
[df.loc[1,i] for i in df.columns] # Each cell contains a single value

['0r7CVbZTWZgbTCYdfa2P31',
 'Memories - Dillon Francis Remix',
 'Maroon 5',
 67,
 '63rPSO264uRjW1X5E6cWv6',
 'Memories (Dillon Francis Remix)',
 '2019-12-13',
 'Pop Remix',
 '37i9dQZF1DXcZDD7cfEKhW',
 'pop',
 'dance pop',
 0.726,
 0.815,
 11,
 -4.968999999999999,
 1,
 0.0373,
 0.0724,
 0.00421,
 0.35700000000000004,
 0.693,
 99.97200000000001,
 162600]

In [5]:
df.shape

(32833, 23)

In [6]:
df.nunique()

track_id                    28356
track_name                  23449
track_artist                10692
track_popularity              101
track_album_id              22545
track_album_name            19743
track_album_release_date     4530
playlist_name                 449
playlist_id                   471
playlist_genre                  6
playlist_subgenre              24
danceability                  822
energy                        952
key                            12
loudness                    10222
mode                            2
speechiness                  1270
acousticness                 3731
instrumentalness             4729
liveness                     1624
valence                      1362
tempo                       17684
duration_ms                 19785
dtype: int64

## 2NF

In [7]:
# Identify candidate Primary Key for each row
# If there is a composite PK, see if other columns have partial dependencies
df_song = df[['track_id', 'track_name', 'track_artist', 'track_popularity', 
              'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 
              'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 
              'duration_ms']]
df_album = df[['track_album_id', 'track_album_name', 'track_album_release_date']]
df_playlist = df[['playlist_id', 'playlist_name', 'playlist_genre', 'playlist_subgenre']]

## 3NF

In [8]:
# Remove transitive dependencies, which means everything should depend on PK directly.

- df_song: track_id, track_name, track_artist, track_popularity and the other song characteristics
- df_album: track_album_id, track_album_name track_album_release_date
- df_playlist_new: playlist_id, playlist_name, playlist_genre
- df_playlist_subgenre: playlist_genre, playlist_subgenre
- df_song_album: track_id and track_album_id
- df_song_playlist: track_id and playlist_id

In [9]:
df_song = df_song.drop_duplicates(subset=['track_id'])
df_album = df_album.drop_duplicates(subset=['track_album_id'])
df_playlist = df_playlist.drop_duplicates(subset=['playlist_id'])

In [10]:
df_playlist_new = df_playlist[['playlist_id', 'playlist_name', 'playlist_genre']]
df_playlist_subgenre = df_playlist[['playlist_genre', 'playlist_subgenre']].drop_duplicates(
    subset=['playlist_genre'])

In [11]:
df_song_album = df[['track_id', 'track_album_id']].drop_duplicates(subset=['track_id'])
df_song_playlist = df[['track_id', 'playlist_id']].drop_duplicates(subset=['track_id'])

# SQLite3 schema

In [12]:
import sqlite3

In [13]:
# Create database
con = sqlite3.connect('spotify.db')

# Create tables in the database
df_song.to_sql(name='song', con=con, index = False)
df_song_album.to_sql(name='song_album', con=con, index = False)
df_album.to_sql(name='album', con=con, index = False)
df_song_playlist.to_sql(name='song_playlist', con=con, index = False)
df_playlist_new.to_sql(name='playlist_new', con=con, index = False)
df_playlist_subgenre.to_sql(name='playlist_subgenre', con=con, index = False)

In [14]:
%load_ext sql

In [15]:
%sql sqlite:///spotify.db

'Connected: @spotify.db'

In [16]:
%%sql
select * from sqlite_master WHERE type='table'

 * sqlite:///spotify.db
Done.


[('table', 'song', 'song', 2, 'CREATE TABLE "song" (\n"track_id" TEXT,\n  "track_name" TEXT,\n  "track_artist" TEXT,\n  "track_popularity" INTEGER,\n  "danceability" REAL,\n  "ener ... (76 characters truncated) ... hiness" REAL,\n  "acousticness" REAL,\n  "instrumentalness" REAL,\n  "liveness" REAL,\n  "valence" REAL,\n  "tempo" REAL,\n  "duration_ms" INTEGER\n)'),
 ('table', 'song_album', 'song_album', 1041, 'CREATE TABLE "song_album" (\n"track_id" TEXT,\n  "track_album_id" TEXT\n)'),
 ('table', 'album', 'album', 1408, 'CREATE TABLE "album" (\n"track_album_id" TEXT,\n  "track_album_name" TEXT,\n  "track_album_release_date" TEXT\n)'),
 ('table', 'song_playlist', 'song_playlist', 1734, 'CREATE TABLE "song_playlist" (\n"track_id" TEXT,\n  "playlist_id" TEXT\n)'),
 ('table', 'playlist_new', 'playlist_new', 2101, 'CREATE TABLE "playlist_new" (\n"playlist_id" TEXT,\n  "playlist_name" TEXT,\n  "playlist_genre" TEXT\n)'),
 ('table', 'playlist_subgenre', 'playlist_subgenre', 2109, 'CREATE TABLE "playlist_subgenre" (\n"playlist_genre" TEXT,\n  "playlist_subgenre" TEXT\n)')]

## SQL query to find the names of all playlists that contain instrumentals

### instrumentalness: 
Predicts whether a track contains no vocals. “Ooh” and “aah” sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly “vocal”. The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content. Values above 0.5 are intended to represent instrumental tracks, but confidence is higher as the value approaches 1.0.

In [17]:
%%sql 
SELECT COUNT(*) AS num_song, playlist_name
FROM song
INNER JOIN song_playlist
ON song_playlist.track_id = song.track_id
INNER JOIN playlist_new
ON playlist_new.playlist_id = song_playlist.playlist_id
WHERE instrumentalness > 0.5 
GROUP BY playlist_name
ORDER BY num_song DESC

 * sqlite:///spotify.db
Done.


[(91, 'Lush Lofi'),
 (88, 'Underground Party | Hypnotic | Minimal | Acid | Big Room | Tech | Liquid'),
 (81, 'Jazz Vibes'),
 (81, 'House Electro 2019'),
 (77, 'Lo-Fi Beats'),
 (75, 'Lofi Hip-Hop'),
 (60, 'Selected House'),
 (58, '💊ELECTRO-HOUSE-TECH💊'),
 (52, 'Tropical Vibes'),
 (50, 'Electropop And Play'),
 (49, 'Electro Posé - Discoveries'),
 (48, 'Techno House 2020 👽 Best Collection 👻 Top DJ’s Electronic Music - Deep House - Trance - Tech House - Dance - Electro Pop'),
 (48, 'Sunny Beats'),
 (44, 'Fitness Workout Electro | House | Dance | Progressive House'),
 (44, 'Deep Electronic Music 2020 & Progressive House'),
 (43, 'House/Electro/Progressive/Disco/Lofi/Synthwave'),
 (41, 'Latin Village 2019'),
 (40, 'Gym (Melbourne Bounce/Progressive House)'),
 (38, 'Nasty Bits'),
 (37, 'Trap Mojito'),
 (37, 'Big Room House | Festival Bangers'),
 (34, 'Jeff Seid Electro House'),
 (32, 'big boom room — TOMORROWLAND EDC EDM BIG ROOM AMF ADE DANCE TRANCE HARDWELL TIESTO'),
 (29, 'Permanent Wave'),
 (29, 'Indie/Jazz Poptimism'),
 (28, 'Gothic / Industrial / Mittelalter / EBM / Futurepop / Gothik / Electropop'),
 (28, 'Bounce United'),
 (28, 'BIG-ROOM NEVER DIES !'),
 (27, 'Arabic Trap'),
 (26, 'Electro/Progressive/Club House'),
 (25, "Electro House - by Spinnin' Records"),
 (24, 'City Pop 1985 シティーポップ'),
 (24, '2011-2014 House'),
 (22, 'Electro House 2020'),
 (22, 'Big Room EDM'),
 (20, 'Music&Other Drugs'),
 (19, 'Progressive Rock / Metal - Rock /Metal  Progresivo'),
 (19, 'Electro Swing'),
 (19, 'Electro House'),
 (19, 'Big Room Beast'),
 (19, 'Big Room 2019'),
 (19, 'Alex Workout'),
 (18, 'Sick Big Room House Drops | EZUMI'),
 (17, 'Epic Bass Drops'),
 (16, 'permanent wave'),
 (15, '🔊BASSBOOSTED🔊⚡ELECTRO HOUSE⚡🔥EDM CAR MUSIC2018/2019🔥'),
 (15, 'Ultimate Indie Presents... Best Indie Tracks of the 2010s'),
 (15, 'Trending EDM by Nik Cooper'),
 (15, 'Festival Music 2019 - Warm Up Music (EDM, Big Room & Progressive House)'),
 (15, 'Electro Swing Top 100'),
 (14, 'Waves Pop and EDM'),
 (14, 'Maxi Pop  GOLD (New Wave, Electropop, Synth Pop...)'),
 (14, 'Hands Up\u200f\u200f\u200b\u200b   \u200d'),
 (14, "Big Room EDM - by Spinnin' Records"),
 (13, 'Trap Nation'),
 (13, '90s Dance Hits'),
 (11, 'Tropical Nights'),
 (11, 'Trance Party 2019 by FUTURE TRANCE'),
 (10, 'Trap Nation 🔊'),
 (10, 'Tastemakers Ball  -  EDM - POP and FUN'),
 (10, 'EDM 2019'),
 (9, 'Vocal House'),
 (9, 'Urban contemporary'),
 (9, 'The Edge of Indie Poptimism'),
 (9, 'ELECTRO HOUSE 2020'),
 (9, 'Badass Rock'),
 (9, 'Afro Psychedelica'),
 (8, '♥ EDM LOVE 2020'),
 (8, 'Vibra Tropical'),
 (8, 'Urban Contemporary'),
 (8, 'Pop Dance Hits'),
 (8, 'PAROOKAVILLE - Big Room'),
 (8, 'NEO SOUL GUITAR'),
 (8, 'NEO FUNK AND SOUL'),
 (8, 'Minitruckin Playlist'),
 (8, 'GTA V - Radio Mirror Park'),
 (8, 'Epic Bass Drops | Best House Mixes'),
 (8, 'Electro Vibes'),
 (8, 'Electro Pop | Electropop'),
 (8, 'Deep-deep Bubble Pop'),
 (8, 'CHRISTIAN ELECTRO / DANCE / EDM'),
 (8, "80's Freestyle/Disco Dance Party (Set Crossfade to 4-Seconds)"),
 (7, '②⓪①⑨ mixed'),
 (7, '[EAS]: Festival Big Room'),
 (7, 'Ultimate Indie Presents... Best Tracks of 2019'),
 (7, 'Pop Warmup 130 BPM'),
 (7, 'Japanese Funk/Soul/NEO/Jazz/Acid'),
 (7, 'Indie Poptimism'),
 (7, 'Hard Rock'),
 (7, 'Electro House Top Tracks'),
 (7, 'Dancefloor Beats'),
 (7, 'Club Mix 2020 🍹'),
 (7, 'Chillout & Remixes 💜'),
 (6, 'Tropical House'),
 (6, 'Permanent wave 🌊'),
 (6, 'Modern Indie Rock // Alternative Rock / Garage Rock / Pop Punk / Grunge / Britpop / Pop Rock'),
 (6, 'INDIE POP! TUNES'),
 (6, 'Hard Rock Workout'),
 (6, 'ELECTROPOP🐹'),
 (6, 'ELECTROPOP EN ESPAÑOL'),
 (6, 'Bluegrass Covers'),
 (6, 'BALLARE - رقص'),
 (6, '@deniceemoberg EDM - POP REMIXES'),
 (6, '70s Hard Rock'),
 (5, 'keg party jukebox'),
 (5, 'indie poptimism'),
 (5, 'The Pulse of Indie Poptimism'),
 (5, 'The 1950s/1960s/1970s/1980s/1990s/2000s/2010s with pop/r&b/soul/boogie/dance/jazz/hip hop/hop/rap.'),
 (5, 'SINULOG Festival 2020'),
 (5, 'POPTIMISM'),
 (5, 'Indie

In [19]:
%%sql
SELECT COUNT(*) AS total_num
FROM(
SELECT COUNT(*) AS num_song, playlist_name
FROM song
INNER JOIN song_playlist
ON song_playlist.track_id = song.track_id
INNER JOIN playlist_new
ON playlist_new.playlist_id = song_playlist.playlist_id
WHERE instrumentalness > 0.5 
GROUP BY playlist_name
ORDER BY num_song DESC
)

 * sqlite:///spotify.db
Done.


[(249,)]